In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data=pd.read_csv('train.csv')
print(data.head())

left=data['left']
train=data.drop(['left'],axis=1)
X_train, X_valid, y_train, y_valid = train_test_split(train, left, train_size=0.8, test_size=0.2,random_state=0)

print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

cols = [col for col in X_train.columns]
print(cols)

     id  emp      comp lastratingdate  left
0  2228  939  phcvroct     17-10-2016     1
1  4349  250  jblrepyr     19-03-2017     0
2   945  134  ewpvmfbc     21-09-2016     0
3  4553  164  wsmblohy     17-03-2017     0
4   941  129  ewpvmfbc     04-04-2016     0
(2820, 4)
(706, 4)
(2820,)
(706,)
['id', 'emp', 'comp', 'lastratingdate']


In [3]:
ratings=pd.read_csv('ratings.csv')
remarks=pd.read_csv('remarks.csv')
remarks_supp_opp=pd.read_csv('remarks_supp_opp.csv')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

print("Ratings: ")
print(ratings.shape)
print(ratings.head())
print("")
print("Remarks: ")
print(remarks.shape)
print(remarks.head())
print("")
print("Remarks_supp_opp: ")
print(remarks_supp_opp.shape)
print(remarks_supp_opp.head())
print("")
print("train: ")
print(train.shape)
print(train.head())
print("")
print("test: ")
print(test.shape)
print(test.head())
#a,b,c=split_date("10-2-2019")

#print(train.iloc[-3:])

/home/sayukta/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Ratings: 
(221232, 4)
   emp      comp        Date  rating
0   31  bnivzbfi  01-02-2016       4
1   33  bnivzbfi  01-02-2016       4
2   79  bnivzbfi  01-02-2016       4
3   94  bnivzbfi  01-02-2016       4
4   16  bnivzbfi  01-02-2016       2

Remarks: 
(82756, 5)
   emp      comp  remarkId                                                txt  \
0  307  bnivzbfi  fvwadfrj  **********************************************...   
1  382  bnivzbfi  xrrfkgap                      *****************************   
2  172  bnivzbfi  xkcrccwi                        ***************************   
3  135  bnivzbfi  lohhvtmo                        ***************************   
4  225  bnivzbfi  gpxxmoab                  *********************************   

   remarkDate  
0  20-03-2017  
1  20-03-2017  
2  20-03-2017  
3  20-03-2017  
4  20-03-2017  

Remarks_supp_opp: 
(336960, 5)
   emp      comp support oppose  remarkId
0  307  bnivzbfi    True  False  fvwadfrj
1   36  bnivzbfi    True  False  fv

In [4]:
def split_date(date):
    #print("date: ",date)
    #print(type(date))
    x=date.split("-")
    return((x[0],x[1],x[2]))

In [ ]:
def sum_str(s):
    a=0
    for i in range(0,len(s)):
        a+=ord(s[i])
    return(a)    

summ=0
X_train=pd.DataFrame(columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp','left'])


# print("train: ")
# print(train)
                   
for i in range(0,train.shape[0]):
    print("i: ",i)
    rating_date_lst=list()      # has dates at which he gave rating as a list
    rating_lst=list()
    remark_date_lst=list()      # has date as a list when he gave remarks
    remark_id_lst=list()        # all the corresponding remark ids
    
    sup_id_lst=list()           # for a given remark id, how many sup he got.
    opp_id_lst=list()
    sup=0
    opp=0
    
    last_r_date, last_r_month, last_r_year=0,0,0
    
    
    emp=train[i:i+1]['emp'].item()   # current employee
    comp=train[i:i+1]['comp'].item() # of current company
    left=train[i:i+1]['left'].item() # has he left the compnay?
    #print(emp)
    #print(type(emp))
    #print(type(comp))
    #a=3
    #print(type(3))
    #print(comp)
    last_rating_date=train[i:i+1]['lastratingdate'].values[0]
    last_r_date, last_r_month, last_r_year=split_date(last_rating_date)

    df=ratings.loc[(ratings['emp']==emp) & (ratings['comp']==comp)]
    #print("ratinggs df: ",df)
    
    rating_date_lst=df['Date'].tolist()  
    #print("rating_date_lst: ",rating_date_lst)
    rating_lst=df['rating'].tolist()
    
    df1=remarks.loc[(remarks['emp']==emp) & (remarks['comp']==comp)]
    #print("remarls df: ",df1)
    remarks_date_lst=df1['remarkDate'].tolist()
    remarks_id_lst=df1['remarkId'].tolist()

    df2=remarks_supp_opp.loc[(remarks_supp_opp['emp']==emp) & (remarks_supp_opp['comp']==comp)]
    
    for h in range(0,len(remarks_id_lst)):
        sup=0
        opp=0
        for k in range(0,df2.shape[0]):
            if(df2['remarkId'].iloc[k]==remarks_id_lst[h]):
                if(df2['support'].iloc[k]==True):
                    sup+=1
                else:
                    opp+=1
        sup_id_lst.append(sup) 
        opp_id_lst.append(opp)
    
    
    print("len(rating_date_lst): ",len(rating_date_lst))
    print("len(remarks_date_lst: )",len(remarks_date_lst))
    print("len(sup_id_lst): ",sup_id_lst)
    print("len(opp_id_lst): ",opp_id_lst)
    print("len(sup_id_lst): ",len(sup_id_lst))
    print("len(opp_id_lst): ",len(opp_id_lst))
    
#     split_rating=[0,int(len(rating_lst)/5),int(2*len(rating_lst)/5),int(3*len(rating_lst)/5),int(4*len(rating_lst)/5),len(rating_lst)]
#     split_remark=[0,int(len(remarks_date_lst)/5),int(2*len(remarks_date_lst)/5),int(3*len(remarks_date_lst)/5),int(4*len(remarks_date_lst)/5),len(remarks_date_lst)]
#     for u in range(0,len(split_rating)-1):
#         for k in range(split_rating[u],split_rating[u+1]):            # for given rating he gave, write that with all the remarks he got.
    ss=max(len(rating_date_lst),len(remarks_date_lst))
    for k in range(0,ss):    
            rating_date_date,rating_date_month,rating_date_year=split_date(rating_date_lst[k%len(rating_date_lst)])
            rated=rating_lst[k%len(rating_lst)] 
            if(len(remarks_date_lst)!=0):
                remarks_date_date,remarks_date_month,remarks_date_year=split_date(remarks_date_lst[k%len(remarks_date_lst)])
                #print("remarks_date_date ",remarks_date_date,"remarks_date_month ",remarks_date_month," remarks_date_year",remarks_date_year)
                dff= pd.DataFrame([[emp,comp,last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_id_lst)],opp_id_lst[k%len(remarks_id_lst)],left]],columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp','left'])
#                 label_encoder = LabelEncoder() 
#                 dff['comp']= label_encoder.fit_transform(dff['comp']) 
                rs=X_train.shape[0]
        
                #X_train.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_id_lst)],opp_id_lst[k%len(remarks_id_lst)],left]
                X_train.loc[rs]=list(dff.iloc[0])
                #print(X_train)
            if(len(remarks_date_lst)==0):
                #remarks_date_date,remarks_date_month,remarks_date_year=split_date(remarks_date_lst[s])
                dff= pd.DataFrame([[emp,comp,last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,0,0,0,0,0,left]],columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp','left'])        
                
                #rs=X_train.shape[0]
                #X_train.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,0,0,0,0,0,left]
#                 label_encoder = LabelEncoder() 
#                 dff['comp']= label_encoder.fit_transform(dff['comp']) 
                rs=X_train.shape[0]
        
                #X_train.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_id_lst)],opp_id_lst[k%len(remarks_id_lst)],left]
                X_train.loc[rs]=list(dff.iloc[0])
                #print(X_train)
    print("--------------------------")
print("X_train: ",X_train) 
print("X_train shape: ",X_train.shape)
label_encoder = LabelEncoder() 
X_train['comp']= label_encoder.fit_transform(X_train['comp']) 

i:  0


/home/sayukta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/sayukta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: `item` has been deprecated and will be removed in a future version
/home/sayukta/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: `item` has been deprecated and will be removed in a future version


len(rating_date_lst):  1
len(remarks_date_lst: ) 0
len(sup_id_lst):  []
len(opp_id_lst):  []
len(sup_id_lst):  0
len(opp_id_lst):  0
--------------------------
i:  1
len(rating_date_lst):  112
len(remarks_date_lst: ) 104
len(sup_id_lst):  [1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  104
len(opp_id_lst):  104
--------------------------
i:  2
len(rating_date_lst):  26
len(remarks_da

--------------------------
i:  33
len(rating_date_lst):  47
len(remarks_date_lst: ) 22
len(sup_id_lst):  [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  22
len(opp_id_lst):  22
--------------------------
i:  34
len(rating_date_lst):  3
len(remarks_date_lst: ) 6
len(sup_id_lst):  [0, 0, 0, 0, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0]
len(sup_id_lst):  6
len(opp_id_lst):  6
--------------------------
i:  35
len(rating_date_lst):  3
len(remarks_date_lst: ) 0
len(sup_id_lst):  []
len(opp_id_lst):  []
len(sup_id_lst):  0
len(opp_id_lst):  0
--------------------------
i:  36
len(rating_date_lst):  12
len(remarks_date_lst: ) 0
len(sup_id_lst):  []
len(opp_id_lst):  []
len(sup_id_lst):  0
len(opp_id_lst):  0
--------------------------
i:  37
len(rating_date_lst):  4
len(remarks_date_lst: ) 2
len(sup_id_lst):  [0, 0]
len(opp_id_lst):  [0, 0]
len(sup_id_lst):  2
len(opp_id_l

--------------------------
i:  67
len(rating_date_lst):  298
len(remarks_date_lst: ) 30
len(sup_id_lst):  [1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  30
len(opp_id_lst):  30
--------------------------
i:  68
len(rating_date_lst):  28
len(remarks_date_lst: ) 38
len(sup_id_lst):  [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  38
len(opp_id_lst):  38
--------------------------
i:  69
len(rating_date_lst):  76
len(remarks_date_lst: ) 56
len(sup_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1

...

--------------------------
i:  2568
len(rating_date_lst):  2
len(remarks_date_lst: ) 0
len(sup_id_lst):  []
len(opp_id_lst):  []
len(sup_id_lst):  0
len(opp_id_lst):  0
--------------------------
i:  2569
len(rating_date_lst):  273
len(remarks_date_lst: ) 182
len(sup_id_lst):  [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

--------------------------
i:  2602
len(rating_date_lst):  19
len(remarks_date_lst: ) 4
len(sup_id_lst):  [0, 0, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0]
len(sup_id_lst):  4
len(opp_id_lst):  4
--------------------------
i:  2603
len(rating_date_lst):  152
len(remarks_date_lst: ) 26
len(sup_id_lst):  [1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  26
len(opp_id_lst):  26
--------------------------
i:  2604
len(rating_date_lst):  18
len(remarks_date_lst: ) 8
len(sup_id_lst):  [0, 0, 1, 1, 0, 0, 1, 1]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  8
len(opp_id_lst):  8
--------------------------
i:  2605
len(rating_date_lst):  1
len(remarks_date_lst: ) 0
len(sup_id_lst):  []
len(opp_id_lst):  []
len(sup_id_lst):  0
len(opp_id_lst):  0
--------------------------
i:  2606
len(rating_date_lst):  58
len(remarks_date_lst: ) 27
len(sup_id_l

--------------------------
i:  2639
len(rating_date_lst):  159
len(remarks_date_lst: ) 140
len(sup_id_lst):  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  140
len(opp_i

--------------------------
i:  2659
len(rating_date_lst):  6
len(remarks_date_lst: ) 4
len(sup_id_lst):  [0, 0, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0]
len(sup_id_lst):  4
len(opp_id_lst):  4
--------------------------
i:  2660
len(rating_date_lst):  184
len(remarks_date_lst: ) 312
len(sup_id_lst):  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

--------------------------
i:  2677
len(rating_date_lst):  54
len(remarks_date_lst: ) 4
len(sup_id_lst):  [1, 0, 1, 0]
len(opp_id_lst):  [0, 0, 0, 0]
len(sup_id_lst):  4
len(opp_id_lst):  4
--------------------------
i:  2678
len(rating_date_lst):  3
len(remarks_date_lst: ) 0
len(sup_id_lst):  []
len(opp_id_lst):  []
len(sup_id_lst):  0
len(opp_id_lst):  0
--------------------------
i:  2679
len(rating_date_lst):  20
len(remarks_date_lst: ) 36
len(sup_id_lst):  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(opp_id_lst):  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
len(sup_id_lst):  36
len(opp_id_lst):  36
--------------------------
i:  2680
len(rating_date_lst):  3
len(remarks_date_lst: ) 2
len(sup_id_lst):  [0, 0]
len(opp_id_lst):  [0, 0]
len(sup_id_lst):  2
len(opp_id_lst):  2
--------------------------
i:  2681
len(rating_date_lst):  8
len(remarks_date_l

In [ ]:
def sum_str(s):
    a=0
    for i in range(0,len(s)):
        a+=ord(s[i])
    return(a)    


summ=0
X_test=pd.DataFrame(columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp'])
#print(X_train)
remark_d_store=list()
remark_m_store=list()
remark_y_store=list()
sup_store=list()
opp_store=list()
                     
for i in range(0,test.shape[0]):
    print("i: ",i)
    rating_date_lst=list()      # has dates at which he gave rating as a list
    rating_lst=list()
    remark_date_lst=list()      # has date as a list when he gave remarks
    remark_id_lst=list()        # all the corresponding remark ids
    
    sup_id_lst=list()           # for a given remark id, how many sup he got.
    opp_id_lst=list()
    sup=0
    opp=0
    
    last_r_date, last_r_month, last_r_year=0,0,0
    
    
    emp=test[i:i+1]['emp'].item()
    comp=test[i:i+1]['comp'].item()
    #print(emp)
    #print(type(emp))
    #print(type(comp))
    #a=3
    #print(type(3))
    #print(comp)
    last_rating_date=test[i:i+1]['lastratingdate'].values[0]
    last_r_date, last_r_month, last_r_year=split_date(last_rating_date)

    df=ratings.loc[(ratings['emp']==emp) & (ratings['comp']==comp)]
    #print("ratinggs df: ",df)
    
    rating_date_lst=df['Date'].tolist()
    rating_lst=df['rating'].tolist()
    
    df1=remarks.loc[(remarks['emp']==emp) & (remarks['comp']==comp)]
    #print("remarls df: ",df1)
    remarks_date_lst=df1['remarkDate'].tolist()
    remarks_id_lst=df1['remarkId'].tolist()

    df2=remarks_supp_opp.loc[(remarks_supp_opp['emp']==emp) & (remarks_supp_opp['comp']==comp)]
    
    for h in range(0,len(remarks_id_lst)):
        sup=0
        opp=0
        for k in range(0,df2.shape[0]):
            if(df2['remarkId'].iloc[k]==remarks_id_lst[h]):
                if(df2['support'].iloc[k]==True):
                    sup+=1
                else:
                    opp+=1
        sup_id_lst.append(sup) 
        opp_id_lst.append(opp)
    
    
    print("len(rating_date_lst): ",len(rating_date_lst))
    print("len(remarks_date_lst: )",len(remarks_date_lst))
    print("len(sup_id_lst): ",sup_id_lst)
    print("len(opp_id_lst): ",opp_id_lst)
    print("len(sup_id_lst): ",len(sup_id_lst))
    print("len(opp_id_lst): ",len(opp_id_lst))
    
    
    ss=max(len(rating_date_lst),len(remarks_date_lst))
    for k in range(0,ss):  
            rating_date_date,rating_date_month,rating_date_year=split_date(rating_date_lst[k%len(rating_date_lst)])
            rated=rating_lst[k%len(rating_lst)]  
        
            if(len(remarks_date_lst)!=0):
                #print("remark date: ",remarks_date_lst[k%len(remarks_date_lst)])
                if(type(remarks_date_lst[k%len(remarks_date_lst)])==float):
                    print("FLOATTT: ",remarks_date_lst[k%len(remarks_date_lst)])
                    dff= pd.DataFrame([[emp,comp,last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_date_lst)],opp_id_lst[k%len(remarks_date_lst)]]],columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp'])

                    rs=X_test.shape[0]
#                     label_encoder = LabelEncoder() 
#                     dff['comp']= label_encoder.fit_transform(dff['comp']) 
                    rs=X_test.shape[0]
        
                    #X_train.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_id_lst)],opp_id_lst[k%len(remarks_id_lst)],left]
                    X_test.loc[rs]=list(dff.iloc[0])  
                    #X_test.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,np.mean(remark_d_store),np.mean(remark_m_store),np.mean(remark_y_store),np.mean(sup_store),np.mean(opp_store)] 
                    continue
                remarks_date_date,remarks_date_month,remarks_date_year=split_date(remarks_date_lst[k%len(remarks_date_lst)])
                
                remark_d_store.append(remarks_date_date)
                remark_m_store.append(remarks_date_month)
                remark_y_store.append(remarks_date_year)
                sup_store.append(sup_id_lst[k%len(remarks_date_lst)])
                opp_store.append(opp_id_lst[k%len(remarks_date_lst)])
                 
                dff= pd.DataFrame([[emp,comp,last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_date_lst)],opp_id_lst[k%len(remarks_date_lst)]]],columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp'])
#                 label_encoder = LabelEncoder() 
#                 dff['comp']= label_encoder.fit_transform(dff['comp']) 
                rs=X_test.shape[0]
        
                #X_train.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_id_lst)],opp_id_lst[k%len(remarks_id_lst)],left]
                X_test.loc[rs]=list(dff.iloc[0])  
                #print("dff: ",dff)
                #rs=X_test.shape[0]
                # Add new row to X_train using loc. Pass the row to be added as a list.
                #X_test.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_date_lst)],opp_id_lst[k%len(remarks_date_lst)]]
            #print("X_train_new: ",X_train)
            if(len(remarks_date_lst)==0):
            #remarks_date_date,remarks_date_month,remarks_date_year=split_date(remarks_date_lst[s])
                dff= pd.DataFrame([[emp,comp,last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,0,0,0,0,0]],columns=['emp','comp','last_rating_date','last_rating_month','last_rating_year','rating_date','rating_month','rating_year','rating','remark_date','remark_month','remark_year','sup','opp'])        
                #print("dffff: ",dff)
                #print("dff shape: ",dff.shape)
                #print("X_train.shape: ",X_train.shape)
                remark_d_store.append(0)
                remark_m_store.append(0)
                remark_y_store.append(0)
                sup_store.append(0)
                opp_store.append(0)
                
                rs=X_test.shape[0]
                #label_encoder = LabelEncoder() 
                #dff['comp']= label_encoder.fit_transform(dff['comp']) 
                #rs=X_test.shape[0]
        
                #X_train.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,remarks_date_date,remarks_date_month,remarks_date_year,sup_id_lst[k%len(remarks_id_lst)],opp_id_lst[k%len(remarks_id_lst)],left]
                X_test.loc[rs]=list(dff.iloc[0])  
                #X_test.loc[rs]=[emp,sum_str(comp),last_r_date,last_r_month,last_r_year,rating_date_date,rating_date_month,rating_date_year,rated,0,0,0,0,0]
                #print("X_train_new: ",X_train)
    print("--------------------------")
print("X_test: ",X_test) 
print("X_test shape: ",X_test.shape)
#label_encoder = LabelEncoder() 
X_test['comp']= label_encoder.transform(X_test['comp']) 

In [88]:
y1=X_train['left']
#y=X_train['left']
X1=X_train.drop(['left'],axis=1)
#X=X_train.drop(['left'],axis=1)


X, X_val, y, y_val = train_test_split(np.array(X1), np.array(y1), train_size=0.8, test_size=0.2, random_state=0)
#X=np.array(X)
#y=np.array(y) 

print(X.shape)
print(y.shape)
# model=XGBClassifier()
# model.fit(X,y)
my_model = XGBRegressor(n_estimators=500, learning_rate=0.05)
my_model.fit(X, y, early_stopping_rounds=5, eval_set=[(X_val, y_val)], verbose=False)


y_pred = my_model.predict(np.array(X_val))
predictions = [round(value) for value in y_pred]

print(y_val.shape)
print(len(predictions))
accuracy = accuracy_score(list(y_val), predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


(148844, 14)
(148844,)
(37212,)
37212
Accuracy: 99.92%


In [99]:
print(X_test)
print(X_train)

y_pred = my_model.predict(np.array(X_test))
predictions = [round(value) for value in y_pred]
#print("y_val: ",y_val)
#print("predictions: ",predictions)
# print(predictions)
# print(len(X_test))
print(len(predictions))

k=0
l=list()

emp=-1
comp=-1

X_test=np.array(X_test)
#print(X_test)

for i in range(0,X_test.shape[0]):
    if(X_test[i][0]!=emp or X_test[i][1]!=comp):
        if(k!=0):
            l.append(k)
        k=1
        emp=X_test[i][0]
        comp=X_test[i][1]
    else:
        k+=1
l.append(k)        
print(l)
print(len(l))
print(sum(l))

ll=list()
zero_cnt=0
curr_index=0

for k in range(0,len(l)):
    s=l[k]
    zero_cnt=0
    for j in range(curr_index,curr_index+s):
        if(predictions[j]==0):
            zero_cnt+=1
    curr_index+=s
    if(zero_cnt>=s/2):
        ll.append(0)
    else:
        ll.append(1)
print(ll)   
print(len(ll))

ll2=list(test['id'].astype('int'))                                     # test['id'] made into int list.   

l_final=list()
for i in range(0,len(ll)):
    l_final.append((ll2[i],ll[i]))

print("l_final_len: ",len(l_final))
csvfile=open('Output2.csv','w', newline='')
obj=csv.writer(csvfile)
for i in range(0,len(l_final)):
    if(i==0):
          obj.writerow(('id','left'))
    if(l_final[i][1]==0):      
        obj.writerow((l_final[i][0],0))
    else:
        obj.writerow((l_final[i][0],1))
csvfile.close()


       emp  comp  last_rating_date  last_rating_month  last_rating_year  \
0      emp  comp  last_rating_date  last_rating_month  last_rating_year   
1      emp  comp  last_rating_date  last_rating_month  last_rating_year   
2      emp  comp  last_rating_date  last_rating_month  last_rating_year   
3      emp  comp  last_rating_date  last_rating_month  last_rating_year   
4      emp  comp  last_rating_date  last_rating_month  last_rating_year   
...    ...   ...               ...                ...               ...   
45943  emp  comp  last_rating_date  last_rating_month  last_rating_year   
45944  emp  comp  last_rating_date  last_rating_month  last_rating_year   
45945  emp  comp  last_rating_date  last_rating_month  last_rating_year   
45946  emp  comp  last_rating_date  last_rating_month  last_rating_year   
45947  emp  comp  last_rating_date  last_rating_month  last_rating_year   

       rating_date  rating_month  rating_year  rating  remark_date  \
0      rating_date  rating_mo

ValueError: could not convert string to float: 'emp'